<a href="https://colab.research.google.com/github/fbrzflrsprcs/AI-2020-V/blob/master/Copia_de_Keras_Flowers_transfer_learning_(solution).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training on GPU will be fine for transfer learning as it is not a very demanding process.

## Imports

In [0]:
import os, sys, math
import numpy as np
from matplotlib import pyplot as plt
if 'google.colab' in sys.modules: # Colab-only Tensorflow version selector
  %tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

TensorFlow 2.x selected.
Tensorflow version 2.1.0


## Configuration

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#@title display utilities [RUN ME]

def dataset_to_numpy_util(dataset, N):
  dataset = dataset.batch(N)
  
  for images, labels in dataset:
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    break;

  return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
  correct = (label == correct_label)
  return "{} [{}{}{}]".format(CLASSES[label], str(correct), ', shoud be ' if not correct else '',
                              CLASSES[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False):
    plt.subplot(subplot)
    plt.axis('off')
    plt.imshow(image)
    plt.title(title, fontsize=16, color='red' if red else 'black')
    return subplot+1
  
def display_9_images_from_dataset(dataset):
  subplot=331
  plt.figure(figsize=(13,13))
  images, labels = dataset_to_numpy_util(dataset, 9)
  for i, image in enumerate(images):
    title = CLASSES[labels[i]]
    subplot = display_one_flower(image, title, subplot)
    if i >= 8:
      break;
              
  plt.tight_layout()
  plt.subplots_adjust(wspace=0.1, hspace=0.1)
  plt.show()
  
def display_9_images_with_predictions(images, predictions, labels):
  subplot=331
  plt.figure(figsize=(13,13))
  classes = np.argmax(predictions, axis=-1)
  for i, image in enumerate(images):
    title, correct = title_from_label_and_target(classes[i], labels[i])
    subplot = display_one_flower(image, title, subplot, not correct)
    if i >= 8:
      break;
              
  plt.tight_layout()
  plt.subplots_adjust(wspace=0.1, hspace=0.1)
  plt.show()
  
def display_training_curves(training, validation, title, subplot):
  if subplot%10==1: # set up the subplots on the first call
    plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
    plt.tight_layout()
  ax = plt.subplot(subplot)
  ax.set_facecolor('#F8F8F8')
  ax.plot(training)
  ax.plot(validation)
  ax.set_title('model '+ title)
  ax.set_ylabel(title)
  ax.set_xlabel('epoch')
  ax.legend(['train', 'valid.'])

## Read images and labels from TFRecords

In [0]:
img_width, img_height = 80, 80
IMAGE_SIZE = [80, 80]
train_data_dir = '/content/drive/My Drive/Colab Notebooks/data/train'
validation_data_dir = '/content/drive/My Drive/Colab Notebooks/data/validation'
nb_train_samples = 560
nb_validation_samples = 120
epochs = 200
batch_size = 50

## training and validation datasets

## Model [WORK WAS REQUIRED HERE]

In [0]:
pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
#pretrained_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
pretrained_model.trainable = False

### QUESTION 1.
model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

94773248/94765736 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 3, 3, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 36866     
Total params: 23,624,578
Trainable params: 36,866
Non-trainable params: 23,587,712
_________________________________________________________________


## Training

In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen  = ImageDataGenerator( rescale=1. / 255)

train_generator = train_datagen.flow_from_directory( train_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='sparse')
validation_generator = test_datagen.flow_from_directory( validation_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='sparse')
model.fit( train_generator, steps_per_epoch=nb_train_samples // batch_size, epochs=epochs,  validation_data=validation_generator, validation_steps=nb_validation_samples // batch_size)

#history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=EPOCHS,validation_data=validation_generator, validation_steps=nb_validation_samples)

Using TensorFlow backend.


Found 560 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 11 steps, validate for 2 steps
Epoch 1/200
11/11 [==============================] - 112s 10s/step - loss: 0.8980 - accuracy: 0.6784 - val_loss: 1.2829 - val_accuracy: 0.5300
Epoch 2/200
11/11 [==============================] - 4s 339ms/step - loss: 0.5944 - accuracy: 0.8235 - val_loss: 0.7707 - val_accuracy: 0.5300
Epoch 3/200
11/11 [==============================] - 2s 162ms/step - loss: 0.3072 - accuracy: 0.8784 - val_loss: 0.9445 - val_accuracy: 0.4700
Epoch 4/200
11/11 [==============================] - 2s 162ms/step - loss: 0.2326 - accuracy: 0.9275 - val_loss: 1.0091 - val_accuracy: 0.4700
Epoch 5/200
11/11 [==============================] - 2s 163ms/step - loss: 0.1960 - accuracy: 0.9333 - val_loss: 1.1000 - val_accuracy: 0.4700
Epoch 6/200
11/11 [==============================] - 2s 177ms/step - loss: 0.2148 - accuracy: 0.9255 - val_loss:

In [0]:
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
img_path = '/content/drive/My Drive/Colab Notebooks/data/validation/class1/1_67_3043537.jpg'
img = image.load_img(img_path, target_size=(80, 80))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
preds = np.argmax(preds, axis=-1)
print(preds)#output [0] significa que pertence a la clase 1

[0]


In [0]:
img_path = '/content/drive/My Drive/Colab Notebooks/data/validation/class2/2_88_81794.jpg'
img = image.load_img(img_path, target_size=(80, 80))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
preds = np.argmax(preds, axis=-1)
print(preds)#output [1] significa que pertence a la clase 2

[1]
